In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
import numpy as np
import tensorflow as tf
import calendar
from write_to_tf import write_tfrecord
# Visualize the result

total cores: 256
1 Physical GPUs, 1 Logical GPUs


2025-07-31 15:57:16.059885: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14678 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


bands=[31]
#max_vals = np.load("%smodels/max_vals_dnb_l95_z50_ps128_(29)_cao_months_202012-202111.npy" %data_loc)
print(len(bands))

#encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")

1 Physical GPUs, 1 Logical GPUs
1


In [ ]:
# Load training data
bands = [31]
patch_size = 128
band_str = "31"
model_run_name = f"dnb_ice01_l95_z50_ps128_band31" # ex.
yearly_data = True

years = [2019, 2020, 2021, 2022, 2023]
months = [1, 2, 3, 4, 10, 11, 12]
train_data = []
test_data = []

data_loc = "/dir_to_your_data/"
model_folder_save = "/dir_to_save_min_and_max/"
model_tfdata_save = "/dir_to_save_tf_data_used_for_model_training/"
os.makedirs(model_folder_save, exist_ok=True)
os.makedirs(model_tfdata_save, exist_ok=True)

# Load numpy arrays containing training and test patches
for year in years:
    print("Year:", year)
    if yearly_data:
        train_data.append(np.load(f"{data_loc}/train_{model_run_name}_{year}.npy"))
        test_data.append(np.load(f"{data_loc}/test_{model_run_name}_{year}.npy"))
    else:
        for month in months:
            _, days_in_month = calendar.monthrange(year, month)
            for day in range(1, days_in_month+1):
                date = str(year) + str(month).zfill(2) + str(day).zfill(2)
                try:
                    train_data.append(np.load(f"{data_loc}/train_{model_run_name}_{date}.npz")["train_patches"])
                    test_data.append(np.load(f"{data_loc}/test_{model_run_name}_{date}.npz")["test_patches"])
                    # if not os.path.exists(f"/scratch/fslippe/modis/training_data/band31/test_{model_run_name}_{date}.npz"):
                    #     print(f"File     not found for date {date}")

                except FileNotFoundError as e:
                    print(f"File not found for date {date}: {e}")
                    continue


Year: 2019
Year: 2020
Year: 2021
Year: 2022
Year: 2023


In [ ]:
# Combine all
train_data = np.concatenate((train_data), axis=0)
test_data = np.concatenate((test_data), axis=0)
max_val = np.max(train_data)
min_val = np.min(train_data)

print("number of patches:", len(train_data))
print("max:", max_val)
print("min:", min_val)

np.save(model_folder_save + "max_val_%s" %(model_run_name) + f"_{np.min(years)}-{np.max(years)}", max_val)
np.save(model_folder_save + "min_val_%s" %(model_run_name) + f"_{np.min(years)}-{np.max(years)}", min_val)

normlized_train_data = (train_data - min_val) / (max_val - min_val)
normlized_test_data = (test_data - min_val) / (max_val - min_val)
np.save(model_tfdata_save + "normalized_valpatches_%s" %(model_run_name), normlized_test_data)

number of patches: 32582
max: 11.319851
min: 1.0262215


In [ ]:
# patches_per_file = int(200000 * (128/patch_size)**2)
patches_per_file = 100000

tot_files = int(len(normlized_train_data) / patches_per_file)

for i in range(tot_files):
    if i < tot_files-1:
        chunk = normlized_train_data[i*patches_per_file: i*patches_per_file+patches_per_file]
    else:
        chunk = normlized_train_data[i*patches_per_file: ]
    print(i, len(chunk))
    print(f"{model_tfdata_save}_{i}.tfrecord")
    write_tfrecord(model_tfdata_save + f"normalized_trainingpatches_{model_run_name}_{i}.tfrecord", chunk)


0 10000
/scratch/fslippe/modis/training_data/tf_data/dnb_l95_z50_ps384_band31/_0.tfrecord
1 10000
/scratch/fslippe/modis/training_data/tf_data/dnb_l95_z50_ps384_band31/_1.tfrecord
2 12582
/scratch/fslippe/modis/training_data/tf_data/dnb_l95_z50_ps384_band31/_2.tfrecord
